# **Serialization and advanced indexing in FAISS**

FAISS provides advanced tools for managing and optimizing indices, enabling us to:
1. Save and load indices using serialization.
2. Merge multiple indices for distributed or incremental indexing.
3. Combine different index types to achieve hybrid search strategies.


In [1]:
import faiss
import numpy as np

## FAISS index serialization
Serialization allows us to save an index to disk and reload it later. This is useful for persisting trained indices for future use, sharing indices between applications or systems and avoiding retraining on the same data.

- FAISS provides functions `faiss.write_index()` and `faiss.read_index()` to save and load indices to/from a file on disk.
- The file extension for the saved index is usually `.faiss` (or any other custom extension), and FAISS can operate on these saved indices by loading them into memory when required.
- The index is stored in a binary format, and FAISS handles the necessary operations to ensure it can be reloaded later in exactly the same state it was when saved.
- FAISS can store the trained index on disk, but it requires vectors to be loaded into memory during the search. This means that the vectors we add to the index need to be in-memory when performing a search. That is different from other vaector databases such as Milvus, that can handle searches without needing the entire dataset to be loaded into memory. It supports both in-memory and disk-based indexing, so the index can stay on disk while being searched efficiently through an optimized data structure (like IVF or PQ).

##### Step 1: Create a random dataset and index


In [2]:
# Generate a dataset of random vectors
dimension = 128  # Each vector has 128 dimensions
num_vectors = 1000  # 1000 vectors
data = np.random.random((num_vectors, dimension)).astype('float32')  # Generate random data

# Create an index and add data
index = faiss.IndexFlatL2(dimension)
index.add(data)

Here, we create a dataset of 1000 random vectors, each with 128 dimensions. This data will be used for indexing and search. FAISS requires data in `float32` format for efficient processing. Then, we create an index using `IndexFlatL2`, which is a brute-force search method based on Euclidean (L2) distance. This type of index compares every vector in the dataset with the query to find the nearest neighbors. The generated dataset (`data`) is added to the index for future searches.

##### Step 2: Save the index to disk

In [3]:
# Save the index to disk
faiss.write_index(index, "flat_index.faiss")
print("Index saved to 'flat_index.faiss'.")

Index saved to 'flat_index.faiss'.


We use `faiss.write_index()` to serialize the index and save it to a file called `"flat_index.faiss"`. This file can now be stored on disk and reloaded later.

##### Step 3: Load the Index from Disk

In [4]:
# Load the index from disk
loaded_index = faiss.read_index("flat_index.faiss")
print("Index loaded from 'flat_index.faiss'.")

Index loaded from 'flat_index.faiss'.


We use `faiss.read_index()` to load the saved index (`"flat_index.faiss"`) back into memory. The index is now ready to perform searches, exactly as it was before it was saved.

##### Step 4: Perform a search on the loaded index

In [5]:
# Verify the loaded index by performing a search
query_vector = np.random.random((1, dimension)).astype('float32')  # Create a random query vector
distances, indices = loaded_index.search(query_vector, 5)  # Search for the top 5 nearest neighbors
print("Search results:", indices)  # Display the indices of the nearest neighbors

Search results: [[733 576 740 907 869]]


After loading the index, we can use it to search for the nearest neighbors of a new `query_vector` (a random vector). We search for the top 5 nearest neighbors using the `search()` method.

## Merging multiple indices
Index merging allows us to combine two or more indices into a single index. This is useful for:
- Distributed indexing: When data is split across multiple machines or locations, we may want to index each subset separately and later merge them into a single index for efficient searching.
- Incremental indexing: If new vectors are added periodically, we can index the new data separately and later merge it into the main index without rebuilding the entire index from scratch.

Merging indices in FAISS essentially consolidates multiple indices, each holding different subsets of data, into a single, unified index that can be used for search operations.

In FAISS, indices can be merged using the `merge_from()` method. This method allows us to merge one index into another so that both indices’ data will be available in the final index. The merging process is typically done without losing any of the data, and it allows us to perform search operations on the merged index, just like we would on a single index.

##### Step 1: Create two datasets with random data


In [6]:
# Step 1: Create two datasets with random data
data1 = np.random.random((500, dimension)).astype('float32')  # 500 vectors for the first dataset
data2 = np.random.random((500, dimension)).astype('float32')  # 500 vectors for the second dataset

We generate two separate datasets, `data1` and `data2`, each containing 500 vectors with a specified `dimension` (the number of features or elements in each vector).

##### Step 2: Create two indices to hold the datasets

In [7]:
# Step 2: Create two indices to hold the datasets
index1 = faiss.IndexFlatL2(dimension)  # First index for data1
index2 = faiss.IndexFlatL2(dimension)  # Second index for data2

We create two separate FAISS indices (`index1` and `index2`) using the IndexFlatL2 class.

##### Step 3: Add vectors to the indices

In [8]:
# Step 3: Add vectors to the indices
index1.add(data1)  # Add the first set of data to index1
index2.add(data2)  # Add the second set of data to index2

We add the two datasets (`data1` and `data2`) to their respective indices using the `add()` method. Each index now holds the vectors from its respective dataset.

##### Step 4: Merge `index2` into `index1`

In [9]:
# Step 4: Merge index2 into index1
index1.merge_from(index2)
print("Number of vectors after merging:", index1.ntotal)

Number of vectors after merging: 1000


The `merge_from()` method is called on `index1`, and `index2` is passed as an argument. This means that all the vectors in `index2` are now merged into `index1`. The result is that `index1` will now contain the combined vectors from both `data1` and `data2`. After merging, the vectors from both datasets are stored in `index1`.

> Note: The indices we are merging must be of the same type. For example, we cannot merge an `IndexFlatL2` with an `IndexIVFFlat` directly. Also, merging indices that use different quantization methods is not supported directly.

##### Step 5: Perform a search on the merged index

In [10]:
query_vector = np.random.random((1, dimension)).astype('float32')  # Create a random query vector
distances, indices = index1.search(query_vector, 5)  # Search for the 5 nearest neighbors
print("Search results:", indices)

Search results: [[495 215 564 749 377]]


We create a random query vector (`query_vector`) and search for the top 5 nearest neighbors in the merged index (`index1`).

## Combining multiple indices
Combining multiple indices involves using different index types or strategies together to create a hybrid search approach. This allows us to leverage the strengths of different indices for improved speed, accuracy, or flexibility when performing searches.

##### **Why Combine Indices?**
1. Speed vs. accuracy: Some indices, like `IndexFlatL2`, provide high accuracy but are slow, while others, like `IndexIVF` (Inverted File Index), are much faster but may introduce some approximation. Combining them allows us to get a balance of both — fast retrieval with good accuracy.
2. Attribute-based partitioning: In some cases, we might want to partition data by attributes (e.g., vectors with different categories or features) and use different index types for different categories. This allows more specialized indexing for specific parts of our data.
3. Distributed indexing: By combining indices into an `IndexShards`, we can distribute the indexing load and allow for parallel searches across multiple indices (or machines) in a scalable manner.

#### Example: Combining flat and IVF indices

In [11]:
# Create a Flat index and an IVF index
index_flat = faiss.IndexFlatL2(dimension)  # Flat index using brute-force search (L2 distance)
index_ivf = faiss.IndexIVFFlat(faiss.IndexFlatL2(dimension), dimension, 10)  # IVF index with 10 clusters

- `index_flat` is a **brute-force index** (IndexFlatL2) that computes exact nearest neighbors using L2 (Euclidean) distance. It’s accurate but slow for large datasets.
- `index_ivf` is an **inverted file index** (IndexIVFFlat) that uses clustering to speed up the search. It divides the data into clusters (in this case, 10 clusters) and only searches the relevant clusters for the query. This reduces the number of comparisons needed, but the results are approximate.

In [12]:
# Train the IVF index and add data
index_ivf.train(data)  # Training the IVF index on the dataset to create clusters
index_ivf.add(data)  # Add data to the IVF index

Before we can add data to the `index_ivf`, we need to train it on the dataset. This step involves computing the centroids of 10 clusters. It’s required for clustering-based indices like IVF. After training, we can add the vectors to the index. The data is then distributed among the 10 clusters based on proximity to the centroids.

In [13]:
# Combine the two indices into an IndexShards
sharded_index = faiss.IndexShards(dimension)  # Create an IndexShards object for combining indices
sharded_index.add_shard(index_flat)  # Add the flat index as one shard
sharded_index.add_shard(index_ivf)   # Add the IVF index as another shard

`IndexShards` is a special type of index that allows us to combine multiple indices. It’s like a container that holds multiple indices and allows us to perform searches across all of them. By adding `index_flat` and `index_ivf` as **shards**, we are telling FAISS to perform searches using both indices together. This is useful when we want to combine the benefits of both types of indices.
- The **flat index** will give us exact, brute-force searches for accuracy.
- The **IVF index** will give us fast, approximate searches using clusters.

We cannot combine indices of different types unless they are compatible. We must ensure that all indices being combined are compatible in terms of how data is stored and how the search is performed.

In [14]:
# Perform a search across both indices
query_vector = np.random.random((1, dimension)).astype('float32')  # Generate a random query vector
distances, indices = sharded_index.search(query_vector, 5)  # Perform a search across the combined indices
print("Indices of nearest neighbors:", indices)
print("Distances to nearest neighbors:", distances)

Indices of nearest neighbors: [[371 965 291 680 915]]
Distances to nearest neighbors: [[16.6833   16.831635 16.843033 16.87417  17.035652]]


We create a query vector (randomly generated in this case) and perform a search on the `sharded_index`. This will search across both the **Flat index** and the **IVF index**. The search returns the top-k nearest neighbors to the query vector. The `sharded_index.search()` will automatically query both indices and return the results from both.
